In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras.layers as layers
from keras.layers import Dense, Conv2D, Activation, MaxPooling2D, Flatten, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau
import math
import albumentations as A

# Model

stworzony na podstawie modelu z poprzednich części zadania.

In [2]:
f_mnist_model = tf.keras.Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(28, 28 ,1)),
    MaxPooling2D(),
    Dropout(0.3),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.5),
    layers.Flatten(),
    layers.Dense(350,activation='relu'),
    layers.Dense(150, activation='relu'),
    Dropout(0.25),
    layers.Dense(10, activation='softmax')
])

f_mnist_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=4, min_lr=0.00000001)

# Przygotowanie danych.
Dane Fashion_Mnist są stworzone następująco: 60 000 przykładów treningowych oraz 10 000 przykładów testowych.
Na zestawie treningowym (60 000) przeprowadzę technikę augmentacji. Stworzę dodatkowe dane (w sumie zbiór danych treningowych będzie 2 razy liczniejszy), które następnie podzielę na zbiór treningowy i walidacyjny w zadanym stosunku 0.9 do 0.1. Na podstawie tych danych wytrenuję model.
Zestaw testowy pozostawiam do testowania wytrenowanego modelu.

In [3]:
def augmentation (data):

    augmented_images = []

    for image in data:
      transform1 = A.HorizontalFlip(p=0.5)
      transform2 = A.RandomRotate90()
      transform3 = A.Transpose()
      transform4 = A.Blur(blur_limit=3)
      transform5 = A.OpticalDistortion()
      transform6 = A.GridDistortion()
      augmented_image = transform1(image=image)['image']
      augmented_image = transform2(image=image)['image']
      augmented_image = transform3(image=image)['image']
      augmented_image = transform4(image=image)['image']
      augmented_image = transform5(image=image)['image']
      augmented_image = transform6(image=image)['image']

      augmented_images.append(augmented_image)
    
    images = np.array(augmented_images)
    return images

In [4]:
train, test = tf.keras.datasets.fashion_mnist.load_data()
images, labels = train
images = images/255.0
labels = labels.astype(np.int32)

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
images_aug = augmentation(images)

images_transformed = np.concatenate((images_aug, images), axis=0)
labels_transformed = np.concatenate((labels, labels), axis=None)

In [6]:
print(images_transformed.shape)
print(labels_transformed.shape)

(120000, 28, 28)
(120000,)


In [7]:
X_train, X_val, y_train, y_val = \
    train_test_split(images_transformed, labels_transformed, test_size=0.1, random_state=4321, stratify=labels_transformed)

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.batch(128)

train_stats = f_mnist_model.fit(train_ds, epochs=85,callbacks=[reduce_lr],validation_data=(X_val, y_val))

Epoch 1/85
844/844 [==============================] - 13s 7ms/step - loss: 0.6842 - accuracy: 0.7425 - val_loss: 0.4249 - val_accuracy: 0.8386 - lr: 0.0010
Epoch 2/85
844/844 [==============================] - 5s 6ms/step - loss: 0.4539 - accuracy: 0.8307 - val_loss: 0.3453 - val_accuracy: 0.8701 - lr: 0.0010
Epoch 3/85
844/844 [==============================] - 5s 6ms/step - loss: 0.3978 - accuracy: 0.8518 - val_loss: 0.3061 - val_accuracy: 0.8826 - lr: 0.0010
Epoch 4/85
844/844 [==============================] - 5s 6ms/step - loss: 0.3630 - accuracy: 0.8643 - val_loss: 0.2785 - val_accuracy: 0.8952 - lr: 0.0010
Epoch 5/85
844/844 [==============================] - 5s 6ms/step - loss: 0.3409 - accuracy: 0.8725 - val_loss: 0.2622 - val_accuracy: 0.9031 - lr: 0.0010
Epoch 6/85
844/844 [==============================] - 5s 6ms/step - loss: 0.3238 - accuracy: 0.8788 - val_loss: 0.2490 - val_accuracy: 0.9062 - lr: 0.0010
Epoch 7/85
844/844 [==============================] - 5s 6ms/step - l

# Dokładność modelu
Na podstawie danych testowych pochodzących z pierwotnego (bez augmentacji) zbioru danych Fashion_Mnist

In [8]:
train, test = tf.keras.datasets.fashion_mnist.load_data()
images, labels = test
images = images/255.0
labels = labels.astype(np.int32)
score = f_mnist_model.evaluate(images, labels)
score

313/313 [==============================] - 1s 3ms/step - loss: 0.2031 - accuracy: 0.9303


[0.20314054191112518, 0.9302999973297119]

# Wnioski:

Uważam, że nie jest źle. Model bywał przetrenowywany, więc w celu złagodzenia tej sytuacji zastosowałam w modelu Dropout().

W internecie (przynajmniej tam, gdzie ja dotarłam) rzadko kto zwraca uwagę na accuracy na danych testowych. Wystarcza, że sam wytrenowany na danych treningowych model ma wysoką dokładność i nie zwraca się uwagi na przetrenowanie modelu. W tych okolicznościach osiągnięcie przez laika (czyli mnie) accuracy ~.93 na danych testowych uważam za sukces.